In [13]:
import paddlehub
paddlehub.server_check()
module = paddlehub.Module(name="emotion_detection_textcnn")

[2021-04-14 17:56:55,343] [    INFO] - Request Hub-Server successfully.
[2021-04-14 17:56:55,415] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


In [14]:
import pandas as pd
import numpy as np

In [177]:
date = str(20160825)

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [178]:
data = pd.read_csv("guba_data/guba_data/guba_by_date/"+date+".csv")

In [201]:
data["predicted"]= np.zeros
data["positive_probs"] = np.zeros
# data["negative_probs"] = np.zeros
# data["neutral_probs"] = np.zeros
for i in range(len(data)):
    if type(data["ct"][i]) == str:
        results = module.emotion_classify(texts = [data["ct"][i]])

    for result in results:
        data["predicted"][i] = result["emotion_label"]
        data['positive_probs'][i] = result['positive_probs']
#         data["negative_probs"][i] = result["negative_probs"]
#         data["neutral_probs"][i] = result["neutral_probs"]

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-201-03bd3365afa0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["predicted"][i] = result["emotion_label"]
<ipython-input-201-03bd3365afa0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['positive_probs'][i] = result['positive_probs']


In [205]:
data.head(5)

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,_id,au,ct,hl,st,predicted,positive_probs
0,201608253079285,一汽領導,一汽集團為何就不知羞恥？\n,一汽集團為何就不知羞恥？,000800.sz,1,0.0038
1,201608253079286,春姑2009,應該是長期利好\n,應該是長期利好,600449.sh,1,0.0092
2,201608253079287,美好儲抱,體育產業多重利好發酵氣爽 布局奧運會概念 氣爽明天建倉，下周收網！\n,體育產業多重利好發酵氣爽 布局奧運會概念 氣爽明天建倉，下周收網！,600343.sh,1,0.0315
3,201608253079288,廣東深圳手機網友,災後材料\n,災後材料,002398.sz,1,0.0087
4,201608253079289,怎知魚之樂123,滬指神龍擺尾，驚天動作翹首以盼方法！八月必備板塊，等待起飛！\n,滬指神龍擺尾，驚天動作翹首以盼方法！八月必備板塊，等待起飛！,300143.sz,1,0.0186


In [206]:
#aggregate the probs by stock ticker
sentiment_sum = pd.pivot_table(data, index="st",values=["positive_probs"], aggfunc=np.sum)
# sentiment_sum = sentiment_sum.reset_index()

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [207]:
recommendation = sentiment_sum.sort_values(by=["positive_probs"],ascending = False)
# sentiment_pos = sentiment_pos.drop(columns = ["negative_probs","neutral_probs","predicted"])
recommendation = recommendation.reset_index()
recommendation.columns = ["buy","positive_prob"]
# sentiment_neg = sentiment_sum.sort_values(by=["negative_probs"],ascending = False)
# sentiment_neg = sentiment_neg.drop(columns = ["positive_probs","neutral_probs","predicted"])
# sentiment_neg = sentiment_neg.reset_index()
# sentiment_neg.columns = ["sell","negative_prob"]
# recommendation = sentiment_pos.join(sentiment_neg)
# recommendation = pd.concat([sentiment_neg,sentiment_pos],keys=index)
# recommendation = sentiment_pos[["st","positive_probs"]] + sentiment_neg[["st","negative_probs"]] 

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [208]:
recommendation.to_csv("recommendation.csv")

In [209]:
stock_list = []
for stock in recommendation["buy"]:
    stock_list.append(stock[0:6])

In [210]:
def stock_date(date):
    stock_date = str(date)[0:4] +"-"+str(date)[4:6]+"-"+str(date)[6:8]
    return stock_date

In [211]:
stock_code = list()
day_return = list()
week_return = list()
month_return = list()
for stock in stock_list:
    try:
        stock_data = pd.read_csv("stock_price/stock_price_original/"+stock+".csv")
    except:
        pass
    for i in range(len(stock_data)):
        if stock_data["date"][i] == stock_date(int(date)):
            stock_code.append(str(stock_data["code"][i]))
            day_return.append(stock_data["1_day_return"][i])
            week_return.append(stock_data["1_week_return"][i])
            month_return.append(stock_data["1_month_return"][i])
    

In [212]:
return_list = [stock_code, day_return, week_return, month_return]
df = pd.DataFrame(columns=['st','day_return','week_return','month_return'])
df['st'] = stock_code
df['day_return'] = day_return
df['week_return'] = week_return
df['month_return'] = month_return
# ,columns=['st','day_return','week_return','month_return'])
df.head()

,st,day_return,week_return,month_return
0,600149,-0.037243,-0.049065,-0.220236
1,600228,0.002428,0.007267,0.145313
2,2312,-0.005455,-0.000907,0.056403
3,601011,0.002628,0.041243,0.090514
4,2695,-0.031769,0.057293,-1.546976


In [213]:
df.to_csv("stock_return.csv")

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
